In [2]:
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import time
import csv
import pandas as pd
import numpy as np

In [3]:
user = "Chris" # Miguel

## Preprocess New York Data

In [4]:
def deNaN(series):
    return series.apply(lambda x: "" if pd.isnull(x) else x)

In [5]:
# import data and create name columns
NY_CD_raw = pd.read_excel("../../Data/Post1790/NY/NY_1790_CD.xlsx",
                      header = 11, usecols = 'H, I, X, Y, AM, AN')
NY_CD_raw.columns = ['First Name', 'Last Name', 'First Name.1', 'Last Name.1',
                     'First Name.2', 'Last Name.2']
NY_CD_raw['Name 1'] =  deNaN(NY_CD_raw['First Name']) + " " + deNaN(NY_CD_raw['Last Name'])
NY_CD_raw['Name 2'] =  deNaN(NY_CD_raw['First Name.1']) + " " + deNaN(NY_CD_raw['Last Name.1'])
NY_CD_raw['Name 3'] =  deNaN(NY_CD_raw['First Name.2']) + " " + deNaN(NY_CD_raw['Last Name.2'])
NY_CD_raw['Name'] = [set([x.replace("  "," ").strip() for x in [name1, name2, name3] if x.strip() != ""]) for name1, name2, name3 in zip(NY_CD_raw['Name 1'], NY_CD_raw['Name 2'], NY_CD_raw['Name 3'])]

In [6]:
# number of names
np.sum(NY_CD_raw['Name'].apply(lambda x: len(x) > 0))

2193

In [7]:
# remove duplicated names
names = NY_CD_raw['Name'][~NY_CD_raw['Name'].astype(str).duplicated(keep='first')]

In [8]:
# cleaning NY names - identify names to be cleaned
names_fix = names.copy()
names_fix = names_fix.apply(lambda set: {el.replace("and Sons", "").replace("and Son", "").replace("and Co", "").strip() for el in set})

name_fix_inds = names_fix[names_fix.astype(str).apply(lambda x:  ' and ' in x)].index
names_fix.loc[name_fix_inds]

1                      {Watson and Greenleaf}
12                {Randall Son and Stenvarts}
20      {Robert Daniel and Guilian Crommelin}
91                 {Brochholz and Livingston}
94                    {Jacob and Philip Mark}
108                      {Anspach and Rogers}
125                {Randall Son and Stewarts}
128                        {Berry and Rogers}
136                {Samuel Ward and Brothers}
220      {H and S Johnson ', H and S Johnson}
322                 {Moses and Charles Ogden}
327               {John and Francis Atkinson}
346                       {H and S Johnson '}
359                     {Lynch and Stoughton}
396           {John and Nicholas J Roosevelt}
414                         {H and S Johnson}
430                 {John and Baron Van Alen}
465                   {Isaac and Henry Truax}
513                {Catherine and Rachel Dow}
724                      {Charles and J Shaw}
725                         {H and S Johnson}
825                    {Pintard an

In [9]:
# split two people with same last name
def simpleSplit(name):
    name2 = name.split(" and ")[1]
    lname2 = name2.split(" ")[-1] if "Van" not in name2 else " ".join(name.split(" ")[-2:])
    name1 = name.split(" and ")[0] + " " + lname2
    return set([name1, name2])

simplefix = ['Jacob and Philip Mark', 'Moses and Charles Ogden', 'John and Francis Atkinson', 'John and Nicholas J Roosevelt',
             'Isaac and Henry Truax', 'Catherine and Rachel Dow', 'Charles and J Shaw', 'Abraham and James Cole',
             'John and James Davenport', 'George and Edm Morewood', 'William and James Constable', 'Horace and Seth Johnston',
             'Michael and Thomas Bull', 'Daniel and E Marsh', 'Michael and Abraham Van Peene', 'John and Alexander Mowatt',
             'Alexander and Michael Mowatt']
names_fix = names_fix.apply(lambda x: simpleSplit(list(x)[0]) if list(x)[0] in simplefix else x)

In [10]:
# change names for companies
company_names = [[set(['Watson and Greenleaf']),  set(['James Greenleaf', 'James Watson'])],
                 [set(['Randall Son and Stenvarts']),  set(['Thomas Randall'])],
                 [set(['Randall Son and Stewarts']), set(['Thomas Randall'])],
                 [set(['Samuel Ward and Brothers']), set(['Samuel Ward'])],
                 [set(['John and Francis Atkinson']), set(['John Atkinson', 'Francis Atkinson'])],
                 [set(['Lynch and Stoughton']), set(['Dominick Lynch', 'Thomas Stoughton'])],
                 [set(['John and Nicholas J Roosevelt']), set(['John Roosevelt', 'Nicholas J Roosevelt'])],
                 [set(['John and Baron Van Alen']), set(['John Van Alen'])],
                 [set(['Pintard and Bleecker']), set(['John Pintard', 'Leonard Bleecker'])],
                 [set(['Le Roy and Bayard']), set(['Herman LeRoy', 'William Bayard'])],
                 [set(['Nicholas Low and H and S Johnson']), set(['Nicholas Low'])],
                 [set(['Robinson and Hartshorn']), set(['Samuel Robinson'])],
                 [set(['Wainwright and Caldwell']), set(['Francis Wainwright', 'Andrew Caldwell'])],
                 [set(['Armstrong and Barnswall']), set(['William Armstrong'])],
                 [set(['Amshong and Barnwall']), set(['William Armstrong'])],
                 [set(['Bleeker and March']), set(['Leonard Bleecker', 'Samuel March'])],
                 [set(['Charles and Vanderborcht and DeWolf']), set(['Charles John Michael De Wolf', 'Henry van der Borcht'])],
                 [set(['Robert Daniel and Guilian Crommelin']), set(['Robert Daniel', 'Guilian Crommelin'])]]
company_names = dict(zip([str(c[0]) for c in company_names], [c[1] for c in company_names]))
names_fix = names_fix.apply(lambda x: company_names.get(str(x), x))

In [11]:
# final name list
name_df = pd.DataFrame(names)
name_df['Fixed Name'] = names_fix
name_df

,Name,Fixed Name
0,{John Delafield},{John Delafield}
1,{Watson and Greenleaf},"{James Watson, James Greenleaf}"
9,{Daniel Crommelin and Sons},{Daniel Crommelin}
10,{William Henderson},{William Henderson}
11,{John O Conner},{John O Conner}
...,...,...
2182,{B Jn A Chevallier},{B Jn A Chevallier}
2183,{Thomas Fitzpatrick},{Thomas Fitzpatrick}
2184,{E Hart},{E Hart}
2187,{Chevallier and Rainctaux},{Chevallier and Rainctaux}


In [12]:
df_list = []

In [13]:
def listPeople(driver, fn, ln, samelocation = False, expandGeography = False, expandNameMatch = False):
    retVal =  ["end"]
    # if no matches found, loosen the restrictions on gegraphy and name
    try:
        count_text = driver.find_element(By.XPATH, "//*[@id=\"results-footer\"]/h3").text
    except:
        # feel free to swap the order, depending on your preferences
        if expandGeography:
            return "geography"
        if expandNameMatch:
            return "name"
        retVal.append("No Match: No Match Found")
        return retVal

    count = int(count_text.split(" of ")[1])
    # if multiple matches found, see if any of them are all in the same place
    # we can categorize this as a "location match"
    if count > 1:
        if samelocation:
            locationlist = []
            if count < 5: # likelihood of same location for over 5 individuals = rare
                for i in range(count):
                    location = driver.find_element(By.XPATH, f"//*[@id=\"sRes-{i}\"]/td[3]").text.split(",")[1]
                    locationlist.append(location)
                if len(set(locationlist)) == 1:
                    name = fn + " " + ln
                    retVal.extend([f"Name: {name}", f"Location: {locationlist[0]}"])
                else:
                    retVal.append("No Match: Too Many Potential Matches Found")
            else:
                retVal.append("No Match: Too Many Potential Matches Found")
        else:
            retVal.append("No Match: Too Many Potential Matches Found")
        return retVal
    # if only one name is found then we categorize this as a person match
    else:
        fname = driver.find_element(By.XPATH, f"//*[@id=\"sRes-0\"]/td[2]/span/span[1]").text
        try:
            lname  = driver.find_element(By.XPATH, f"//*[@id=\"sRes-0\"]/td[2]/span/span[2]").text
        except:
            try:
                lname = driver.find_element(By.XPATH, f"//*[@id=\"sRes-0\"]/td[2]/span/span/span[2]").text
            except:
                lname = driver.find_element(By.XPATH, "//*[@id=\"sRes-0\"]/td[2]/span/span/text()").text
        name = fname + " " + lname
        location = driver.find_element(By.XPATH, f"//*[@id=\"sRes-0\"]/td[3]").text
        slavecount = driver.find_element(By.XPATH, f"//*[@id=\"sRes-0\"]/td[4]").text
        familysize = driver.find_element(By.XPATH, f"//*[@id=\"sRes-0\"]/td[5]").text
        # print output
        retVal.extend([f"Name: {name}", f"Location: {location}", f"Family Size: {familysize}", f"Slavecount: {slavecount}"])
        return retVal

In [14]:
def findMatches(fn, ln, driver, locationmatchlist , namematchlist):
    locationmatchind = 0
    namematchind = 0
    geobool = True
    namebool = True
    # navigate to original url
    og_url = f"https://www-ancestrylibrary-com.proxy.uchicago.edu/search/collections/5058/?name={fn}_{ln}&name_x={namematchlist[namematchind]}&residence=_new+york-usa_35&residence_x=_{locationmatchlist[locationmatchind]}"
    driver.get(og_url)
    time.sleep(1)

    # see if there are any matches using initial strict settings
    val = listPeople(driver, fn, ln, samelocation = True, expandGeography = geobool, expandNameMatch = namebool)
    # if we have found a match or we have exhausted all possible geo/name restriction loosenings, end search
    while type(val) != list and (geobool or namebool):
        # command to do a geographic expansion
        if val == "geography":
            locationmatchind+=1
            if locationmatchind < len(locationmatchlist):
                # navigate to url with new settings
                og_url = f"https://www-ancestrylibrary-com.proxy.uchicago.edu/search/collections/5058/?name={fn}_{ln}&name_x={namematchlist[namematchind]}&residence=_new+york-usa_35&residence_x=_{locationmatchlist[locationmatchind]}"
                driver.get(og_url)
                time.sleep(1)
            else:
                # if we have exhausted all options, prevent future geographic expansions
                locationmatchind -= 1 # reset it to most loose setting
                geobool = False
                val = listPeople(driver, fn, ln, samelocation = True, expandGeography = geobool, expandNameMatch = namebool)
        # command to do name expansion
        elif val == "name":
            namematchind+=1
            if namematchind < len(namematchlist):
                # navigate to url with new settings
                og_url = f"https://www-ancestrylibrary-com.proxy.uchicago.edu/search/collections/5058/?name={fn}_{ln}&name_x={namematchlist[namematchind]}&residence=_new+york-usa_35&residence_x=_{locationmatchlist[locationmatchind]}"
                driver.get(og_url)
                time.sleep(1)
            else:
                # if we have exhausted all options, prevent future name expansions
                namematchind -= 1 # reset it to most loose setting
                namebool = False
                val = listPeople(driver, fn, ln, samelocation = True, expandGeography = geobool, expandNameMatch = namebool)
    val.extend([og_url])
    return val

In [15]:
def parseScrapeResult(res, i):
    remdata = []
    if len(res) == 3:
        matchstatus = res[1].split(": ")[0]
        matchreason = res[1].split(": ")[1]
        remdata.extend([matchstatus, matchreason, np.nan, np.nan, np.nan, res[2]])
    elif len(res) == 4:
        matchstatus = "Match"
        matchreason = "Only Location Found"
        location = res[2].split(": ")[1]
        remdata.extend([matchstatus, matchreason, location, np.nan, np.nan, res[3]])
    else:
        matchstatus = "Match"
        matchreason = "Full Match"
        location = res[2].split(": ")[1]
        familycnt = res[3].split(": ")[1]
        slavecnt = res[4].split(": ")[1]
        remdata.extend([matchstatus, matchreason, location, familycnt, slavecnt, res[5]])
    data = [index, og_name, name, i]
    data.extend(remdata)
    return data

In [23]:
assert(user == "Chris")
driver = webdriver.Safari(executable_path=r'/usr/bin/safaridriver')#set driver
file = pd.read_csv('~/Desktop/login_details.txt')
username = file.columns[0]
password = file[username].tolist()[0]
driver.set_window_size(1400,1000)

driver.get("https://www-ancestrylibrary-com.proxy.uchicago.edu/search/collections/5058/")#get website from nyu database

WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, "okta-signin-username"))).send_keys(username)
time.sleep(1)
driver.find_element(by=By.ID, value="okta-signin-password").send_keys(password)
time.sleep(1)
driver.find_element(by=By.ID, value="okta-signin-submit").click() #sign in

WebDriverWait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH,"//*[@id=\"form62\"]/div/div[2]/iframe")))
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//*[@id=\"auth_methods\"]/fieldset/div[1]/button"))).click()

time.sleep(17)

# specify namematch and locationmatch options
locationmatchlist = ["1-0"]#, "1-0-a"] #1-1 is entire US
namematchlist = ["1_1", "s_s", "ps_ps"] #psi-ps also includes flexibility for initials
for index in name_df.index:
    if index % 100 == 0:
        df = pd.DataFrame(columns = ['Index', 'Original Name', 'Search Name', 'Name #', 'Match Status', 'Match Reason', 'Location', 'Family Size', 'Slavecount', 'url'],
                          data = df_list)
        df.to_csv('NY_results.csv')
    if index > 805:
        og_name = list(name_df.loc[index]['Name'])
        fixed_name = list(name_df.loc[index]['Fixed Name'])
        # distinguish between names that correspond to multiple real names vs. just one name
        if len(fixed_name)>1:
            print(f"Original Name: {og_name}, Search Names:{fixed_name}, Index: {index}")
            i = 0
            for name in fixed_name:
                i += 1
                print(f"Name {i}: {name}")
                fn = " ".join(name.split(" ")[:-1])  if "Van" not in name else " ".join(name.split(" ")[:-2])
                ln = name.split(" ")[-1] if "Van" not in name else " ".join(name.split(" ")[-2:])
                res = findMatches(fn, ln, driver, locationmatchlist , namematchlist)
                data = parseScrapeResult(res, i)
                df_list.append(data)
                print([val + ": " + str(d) for val, d in zip(df.columns, data)])
                print("")
        else:
            print(f"Original Name: {og_name}, Search Name:{fixed_name}, Index: {index}")
            name = fixed_name[0]
            fn = " ".join(name.split(" ")[:-1])  if "Van" not in name else " ".join(name.split(" ")[:-2])
            ln = name.split(" ")[-1] if "Van" not in name else " ".join(name.split(" ")[-2:])
            res = findMatches(fn, ln, driver, locationmatchlist , namematchlist)
            data = parseScrapeResult(res, 1)
            df_list.append(data)
            print([val + ": " + str(d) for val, d in zip(df.columns, data)])
            print("")

/var/folders/5s/dvrxt95949x1pm_sjxm85lj00000gn/T/ipykernel_43428/1497075713.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_list)
/var/folders/5s/dvrxt95949x1pm_sjxm85lj00000gn/T/ipykernel_43428/1497075713.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_list)
/var/folders/5s/dvrxt95949x1pm_sjxm85lj00000gn/T/ipykernel_43428/1497075713.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_list)


Original Name: ['Josiah Blakeley'], Search Name:['Josiah Blakeley'], Index: 807
['Index: 807', "Original Name: ['Josiah Blakeley']", 'Search Name: Josiah Blakeley', 'Name #: 1', 'Match Status: Match', 'Match Reason: Full Match', 'Location: New York City East Ward, New York, New York', 'Family Size: 2', 'Slavecount: \xa0', 'url: https://www-ancestrylibrary-com.proxy.uchicago.edu/search/collections/5058/?name=Josiah_Blakeley&name_x=ps_ps&residence=_new+york-usa_35&residence_x=_1-0']

Original Name: ['John Murray'], Search Name:['John Murray'], Index: 808
['Index: 808', "Original Name: ['John Murray']", 'Search Name: John Murray', 'Name #: 1', 'Match Status: No Match', 'Match Reason: Too Many Potential Matches Found', 'Location: nan', 'Family Size: nan', 'Slavecount: nan', 'url: https://www-ancestrylibrary-com.proxy.uchicago.edu/search/collections/5058/?name=John_Murray&name_x=1_1&residence=_new+york-usa_35&residence_x=_1-0']

Original Name: ['Richard Duyree'], Search Name:['Richard Duyre

/var/folders/5s/dvrxt95949x1pm_sjxm85lj00000gn/T/ipykernel_43428/1497075713.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_list)


['Index: 1000', "Original Name: ['Jesse Selleck']", 'Search Name: Jesse Selleck', 'Name #: 1', 'Match Status: No Match', 'Match Reason: No Match Found', 'Location: nan', 'Family Size: nan', 'Slavecount: nan', 'url: https://www-ancestrylibrary-com.proxy.uchicago.edu/search/collections/5058/?name=Jesse_Selleck&name_x=ps_ps&residence=_new+york-usa_35&residence_x=_1-0']

Original Name: ['James McEvers'], Search Name:['James McEvers'], Index: 1002
['Index: 1002', "Original Name: ['James McEvers']", 'Search Name: James McEvers', 'Name #: 1', 'Match Status: No Match', 'Match Reason: Too Many Potential Matches Found', 'Location: nan', 'Family Size: nan', 'Slavecount: nan', 'url: https://www-ancestrylibrary-com.proxy.uchicago.edu/search/collections/5058/?name=James_McEvers&name_x=1_1&residence=_new+york-usa_35&residence_x=_1-0']

Original Name: ['Uriah Rogers'], Search Name:['Uriah Rogers'], Index: 1003
['Index: 1003', "Original Name: ['Uriah Rogers']", 'Search Name: Uriah Rogers', 'Name #: 1',

/var/folders/5s/dvrxt95949x1pm_sjxm85lj00000gn/T/ipykernel_43428/1497075713.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_list)


...

['Index: 1900', "Original Name: ['Thomas Palmer']", 'Search Name: Thomas Palmer', 'Name #: 1', 'Match Status: No Match', 'Match Reason: Too Many Potential Matches Found', 'Location: nan', 'Family Size: nan', 'Slavecount: nan', 'url: https://www-ancestrylibrary-com.proxy.uchicago.edu/search/collections/5058/?name=Thomas_Palmer&name_x=1_1&residence=_new+york-usa_35&residence_x=_1-0']

Original Name: ['Jn H Cazenover and Co'], Search Name:['Jn H Cazenover'], Index: 1901
['Index: 1901', "Original Name: ['Jn H Cazenover and Co']", 'Search Name: Jn H Cazenover', 'Name #: 1', 'Match Status: No Match', 'Match Reason: No Match Found', 'Location: nan', 'Family Size: nan', 'Slavecount: nan', 'url: https://www-ancestrylibrary-com.proxy.uchicago.edu/search/collections/5058/?name=Jn H_Cazenover&name_x=ps_ps&residence=_new+york-usa_35&residence_x=_1-0']

Original Name: ['Amshong and Barnwall'], Search Name:['William Armstrong'], Index: 1902
['Index: 1902', "Original Name: ['Amshong and Barnwall']", '

/var/folders/5s/dvrxt95949x1pm_sjxm85lj00000gn/T/ipykernel_43428/1497075713.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_list)


['Index: 2000', "Original Name: ['Isaac Richards']", 'Search Name: Isaac Richards', 'Name #: 1', 'Match Status: No Match', 'Match Reason: No Match Found', 'Location: nan', 'Family Size: nan', 'Slavecount: nan', 'url: https://www-ancestrylibrary-com.proxy.uchicago.edu/search/collections/5058/?name=Isaac_Richards&name_x=ps_ps&residence=_new+york-usa_35&residence_x=_1-0']

Original Name: ['John Targe'], Search Name:['John Targe'], Index: 2001
['Index: 2001', "Original Name: ['John Targe']", 'Search Name: John Targe', 'Name #: 1', 'Match Status: Match', 'Match Reason: Full Match', 'Location: New York City East Ward, New York, New York', 'Family Size: 6', 'Slavecount: \xa0', 'url: https://www-ancestrylibrary-com.proxy.uchicago.edu/search/collections/5058/?name=John_Targe&name_x=ps_ps&residence=_new+york-usa_35&residence_x=_1-0']

Original Name: ['Leffingwell Pierpont'], Search Name:['Leffingwell Pierpont'], Index: 2002
['Index: 2002', "Original Name: ['Leffingwell Pierpont']", 'Search Name:

/var/folders/5s/dvrxt95949x1pm_sjxm85lj00000gn/T/ipykernel_43428/1497075713.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_list)


['Index: 2100', "Original Name: ['John Glover']", 'Search Name: John Glover', 'Name #: 1', 'Match Status: No Match', 'Match Reason: Too Many Potential Matches Found', 'Location: nan', 'Family Size: nan', 'Slavecount: nan', 'url: https://www-ancestrylibrary-com.proxy.uchicago.edu/search/collections/5058/?name=John_Glover&name_x=1_1&residence=_new+york-usa_35&residence_x=_1-0']

Original Name: ['Jacques de Volder'], Search Name:['Jacques de Volder'], Index: 2103
['Index: 2103', "Original Name: ['Jacques de Volder']", 'Search Name: Jacques de Volder', 'Name #: 1', 'Match Status: No Match', 'Match Reason: No Match Found', 'Location: nan', 'Family Size: nan', 'Slavecount: nan', 'url: https://www-ancestrylibrary-com.proxy.uchicago.edu/search/collections/5058/?name=Jacques de_Volder&name_x=ps_ps&residence=_new+york-usa_35&residence_x=_1-0']

Original Name: ['Samuel Boyd'], Search Name:['Samuel Boyd'], Index: 2105
['Index: 2105', "Original Name: ['Samuel Boyd']", 'Search Name: Samuel Boyd', 'N

In [33]:
df.loc[df['Slavecount'].apply(lambda x: x == '\xa0').index, 'Slavecount'] = np.nan

In [88]:
df = pd.DataFrame(columns = ['Index', 'Original Name', 'Search Name', 'Name #', 'Match Status', 'Match Reason', 'Location', 'Family Size', 'Slavecount', 'url'],
                  data = df_list)
df.to_csv('NY_results.csv')

In [231]:
df['Original Name2'] = df['Original Name'].apply(lambda x: str(x))

In [232]:
str_convert = dict(zip(df['Original Name2'], df['Original Name']))

In [244]:
df_reformat = df.groupby(['Index', 'Original Name2'])['Search Name'].unique().reset_index()
for col in ['Match Status', 'Match Reason', 'Location', 'Family Size', 'Slavecount', 'url']:
    df_reformat = pd.merge(df_reformat, df.groupby(['Index', 'Original Name2'])[col].unique().reset_index())
df_reformat['Original Name2'] = df_reformat['Original Name2'].apply(lambda x: str_convert[x])
df_reformat.rename({'Original Name2':'Original Name'}, axis = 1, inplace = True)

In [245]:
# change data from list to string
df_reformat['Name Count'] = df_reformat['Search Name'].apply(lambda x: len(x))
for i in range(2):
    df_reformat['Match Status '+str(i+1)] = df_reformat['Match Status'].apply(lambda x: x[i] if len(x) > i else np.nan)
    df_reformat['Match Reason '+str(i+1)] = df_reformat['Match Reason'].apply(lambda x: x[i] if len(x) > i else np.nan)
    df_reformat['Location '+str(i+1)] = df_reformat['Location'].apply(lambda x: x[i] if len(x) > i else np.nan)
    df_reformat['Family Size '+str(i+1)] = df_reformat['Family Size'].apply(lambda x: x[i] if len(x) > i else np.nan)
    df_reformat['Slavecount '+str(i+1)] = df_reformat['Slavecount'].apply(lambda x: x[i] if len(x) > i else np.nan)
    df_reformat['url '+str(i+1)] = df_reformat['url'].apply(lambda x: x[i] if len(x) > i else np.nan)
df_reformat.drop(['Match Status', 'Match Reason', 'Location', 'Family Size', 'Slavecount', 'url'], axis = 1, inplace = True)
# reorder columns
df_reformat = df_reformat[df_reformat.columns[:4].tolist() + [x for x in sorted(df_reformat.columns[4:]) if x not in ['Name 2']]]

In [246]:
for col in ['Match Status', 'Match Reason', 'Location', 'Family Size', 'Slavecount', 'url']:
    df_reformat[col + ' 2'] = [np.nan if col1 == col2 else col2 for col1, col2 in zip(df_reformat[col + ' 1'], df_reformat[col + ' 2'])]

In [247]:
df_reformat['Original Name'] = df_reformat['Original Name'].apply(lambda x: set(x))
df_reformat['Original Name'] = df_reformat['Original Name'].apply(lambda x: str(sorted(x)))
df_reformat

,Index,Original Name,Search Name,Name Count,Family Size 1,Family Size 2,Location 1,Location 2,Match Reason 1,Match Reason 2,Match Status 1,Match Status 2,Slavecount 1,Slavecount 2,url 1,url 2
0,0,['John Delafield'],[John Delafield],1,13,NaN,"New York City East Ward, New York, New York",NaN,Full Match,NaN,Match,NaN,2,NaN,https://www-ancestrylibrary-com.proxy.uchicago...,NaN
1,1,['Watson and Greenleaf'],"[James Watson, James Greenleaf]",2,NaN,NaN,NaN,NaN,Too Many Potential Matches Found,No Match Found,No Match,NaN,NaN,NaN,https://www-ancestrylibrary-com.proxy.uchicago...,https://www-ancestrylibrary-com.proxy.uchicago...
2,9,['Daniel Crommelin and Sons'],[Daniel Crommelin],1,NaN,NaN,NaN,NaN,No Match Found,NaN,No Match,NaN,NaN,NaN,https://www-ancestrylibrary-com.proxy.uchicago...,NaN
3,10,['William Henderson'],[William Henderson],1,NaN,NaN,NaN,NaN,Too Many Potential Matches Found,NaN,No Match,NaN,NaN,NaN,https://www-ancestrylibrary-com.proxy.uchicago...,NaN
4,11,['John O Conner'],[John O Conner],1,NaN,NaN,NaN,NaN,Too Many Potential Matches Found,NaN,No Match,NaN,NaN,NaN,https://www-ancestrylibrary-com.proxy.uchicago...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
918,2182,['B Jn A Chevallier'],[B Jn A Chevallier],1,NaN,NaN,New York,NaN,Only Location Found,NaN,Match,NaN,NaN,NaN,https://www-ancestrylibrary-com.proxy.uchicago...,NaN
919,2183,['Thomas Fitzpatrick'],[Thomas Fitzpatrick],1,NaN,NaN,NaN,NaN,No Match Found,NaN,No Match,NaN,NaN,NaN,https://www-ancestrylibrary-com.proxy.uchicago...,NaN
920,2184,['E Hart'],[E Hart],1,NaN,NaN,NaN,NaN,No Match Found,NaN,No Match,NaN,NaN,NaN,https://www-ancestrylibrary-com.proxy.uchicago...,NaN
921,2187,['Chevallier and Rainctaux'],[Chevallier and Rainctaux],1,NaN,NaN,NaN,NaN,No Match Found,NaN,No Match,NaN,NaN,NaN,https://www-ancestrylibrary-com.proxy.uchicago...,NaN


In [248]:
#prepare loan dataset
NY_CD_raw = pd.read_excel("../../Data/Post1790/NY/NY_1790_CD.xlsx",
                      header = 11, usecols = 'H, I, M, N, X, Y, AC, AD, AM, AN, AR, AS')
NY_CD_raw.columns = ['First Name', 'Last Name', '6p_Dollar', '6p_Cents',
                 'First Name.1', 'Last Name.1', '6p_def_Dollar', '6p_def_Cents',
                 'First Name.2', 'Last Name.2', '3p_Dollar', '3p_Cents']
NY_CD_raw['Name 1'] =  deNaN(NY_CD_raw['First Name']) + " " + deNaN(NY_CD_raw['Last Name'])
NY_CD_raw['Name 2'] =  deNaN(NY_CD_raw['First Name.1']) + " " + deNaN(NY_CD_raw['Last Name.1'])
NY_CD_raw['Name 3'] =  deNaN(NY_CD_raw['First Name.2']) + " " + deNaN(NY_CD_raw['Last Name.2'])
NY_CD_raw['Name'] = [set([x.replace("  "," ").strip() for x in [name1, name2, name3] if x.strip() != ""]) for name1, name2, name3 in zip(NY_CD_raw['Name 1'], NY_CD_raw['Name 2'], NY_CD_raw['Name 3'])]
NY_CD_raw['Name_str'] = NY_CD_raw['Name'].apply(lambda x: str(sorted(x)))
NY_dict = dict(zip(NY_CD_raw['Name_str'], NY_CD_raw['Name']))

In [273]:
# aggregate our reults
NY_CD_raw['6p_Total'] = NY_CD_raw['6p_Dollar'].fillna(0) + NY_CD_raw['6p_Cents'].fillna(0)
NY_CD_raw['6p_def_Total'] = NY_CD_raw['6p_def_Dollar'].fillna(0) + NY_CD_raw['6p_def_Cents'].fillna(0)
NY_CD_raw['3p_Total'] = NY_CD_raw['3p_Dollar'].fillna(0) + NY_CD_raw['3p_Cents'].fillna(0)
NY_grouped = NY_CD_raw.groupby('Name_str')['6p_Total', '6p_def_Total', '3p_Total'].sum().reset_index()
NY_final = pd.merge(NY_grouped, df_reformat, left_on='Name_str', right_on = 'Original Name')
NY_final['Original Name'] = NY_final['Original Name'].apply(lambda x: NY_dict[x])

/var/folders/5s/dvrxt95949x1pm_sjxm85lj00000gn/T/ipykernel_43428/1699274805.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  NY_grouped = NY_CD_raw.groupby('Name_str')['6p_Total', '6p_def_Total', '3p_Total'].sum().reset_index()


In [274]:
#final datacleaning
NY_final.drop(['Name_str', 'Index'], axis = 1, inplace = True)
NY_final['Slavecount 1'] = NY_final['Slavecount 1'].apply(lambda x: np.nan if x == '\xa0' else x)
NY_final['Slavecount 2'] = NY_final['Slavecount 2'].apply(lambda x:  np.nan if x == '\xa0' else x)

In [275]:
NY_final = NY_final[['Original Name', 'Search Name'] + NY_final.columns[:3].tolist() + NY_final.columns[5:].tolist()]

In [276]:
NY_final.rename({'Search Name': 'Search Names'}, axis = 1, inplace = True)

In [280]:
NY_final.sort_values(['6p_Total', '3p_Total'], ascending = False, inplace = True)

In [281]:
NY_final.reset_index(drop = True, inplace = True)
NY_final.to_csv('NY_results_final.csv')